# Export single-cell analyses

This notebook is designed to extract the relevant single-cell information from the z-tracks and plot as a pandas dataframe/csv/excel format for others to interrogate.

In [7]:
import btrack
import pandas as pd
import numpy as np
import glob
import os
import fnmatch
from homuncu_loc import dataio
from tqdm.notebook import tqdm
import warnings

### Find H5 files will single-cell tracks in

In [27]:
# replace the path with your own, think it will be 'Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_dir/' 
# but this is from memory so just check where you saved out the final tracks in previous notebook
h5_fns = dataio.find_h5_files('/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output') 
h5_fns

['/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5',
 '/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231127_20X_23-03-072A5_DAPI_ZO-1_CD16_Mtb_ATG14KO_2hpi_Multichannel Z-Stack_20231127_1551_mphi_gt_masks.h5',
 '/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1558_mphi_gt_masks.h5',
 '/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A4_DAPI_ZO-1_CD16_Mtb_WT_48hpi_Multichannel Z-Stack_20231128_1566_gt_mphi_masks.h5',
 '/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A4_DAPI_ZO-1_CD16_Mtb_WT_48hpi_Multichannel Z-Stack_20231128_1571_mphi_gt_masks.h5',
 '/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_ou

##### Optionally filter h5 filenames to only include a subset

In [4]:
h5_fns =  [fn for fn in h5_fns if any(substring in fn for substring in ['194', '195', '196'])] #[fn for fn in h5_fns if fn.endswith('sc_data.h5') and 'run1' in fn]
h5_fns

[]

### Load tracks

In [17]:
### define parameters to plot in table
cols = ['Cell ID', 
        f'Max. int. ch0', 
        f'Max. int. ch1', 
        f'Max. int. ch2', 
        f'Max. int. ch3', 
        'Mtb+', 
        'N_frames Mtb+', 
        'Max. Mtb area', 
        'Average X', 
        'Average Y'
        ]

In [28]:
sum = 0
error_fns = list()
for tracks_fn in h5_fns:
    with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as hdf:
        try:
            tracks = hdf.tracks #[t for t in hdf.tracks if len(t) > 3]
        except:
            error_fns.append(tracks_fn)
    # now plot into dataframe
    df = pd.DataFrame([(track.ID,
                        np.nanmax(track['mean_intensity'][:,0]),
                        np.nanmax(track['mean_intensity'][:,1]),
                        np.nanmax(track['mean_intensity'][:,2]),
                        np.nanmax(track['mean_intensity'][:,3]),
                        True if np.nansum(track.properties['mtb_status']) > 0 else False,
                        np.nansum(track.properties['mtb_status']),
                        np.nanmax(track.properties['mtb_area']),
                        np.mean(track.x),
                        np.mean(track.y))
                       for track in tracks],
                      columns=cols)
    # create fn
    output_fn = tracks_fn.replace('.h5', 'sc_data.csv')
    if output_fn == tracks_fn:
        warnings.warn("Warning: The output file name is the same as the tracks file name. This will result in overwriting the tracks file.")
    # save out
    df.to_csv(output_fn)
    # check on the fly
    print(tracks_fn, '\n', df.head())
    # how many sc measreuments
    sum += len(df)

[INFO][2023/12/21 02:14:45 PM] Opening HDF file: /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5...
[ERROR][2023/12/21 02:14:45 PM] Tracks not found in /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5
[INFO][2023/12/21 02:14:45 PM] Closing HDF file: /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5


NameError: name 'tracks' is not defined

In [29]:
tracks_fn

'/Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5'

In [32]:
with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as hdf:
    tracks = hdf.tracks#[t for t in hdf.tracks if len(t) > 3]

[INFO][2023/12/21 02:16:19 PM] Opening HDF file: /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5...
[ERROR][2023/12/21 02:16:19 PM] Tracks not found in /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5
[INFO][2023/12/21 02:16:19 PM] Closing HDF file: /Users/lukc/Analysis/homuncu_loc/segmentation_and_tracking/temp_mask_output/20231128_20X_23-03-072A3_DAPI_ZO-1_CD16_Mtb_WT_2hpi_Multichannel Z-Stack_20231128_1557_mphi_gt_masks.h5
